### Created by: Rodrigo Didier, 01/31/21.

# Import Libs

In [ ]:

import os
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split

# DATA SCIENCE PIPLINE --------------------------------------------------------------------------

# 0. Feature Engineering

# 1. Data Extraction

In [ ]:
class dataExtractor:

    def __init__(self):
        self.data_path = os.getenv('DATASET_PATH')
        self.data = self.extractData()

    def extractData(self):
        '''
        Loads a dataset with product data from a specified path.
        '''

        return pd.read_csv(self.data_path)

#  2. Data Formatting

In [ ]:
dataFormatter().categoryToDummy().head()

In [ ]:
dataFormatter().split_train_test_validate()

In [ ]:
class dataFormatter(dataExtractor):
    def __init__(self):
        self.extracted_data = dataExtractor().data
        #self.train_data = self.formatData()[0]
        #self.validation_data = self.formatData()[1]
        
    def formatData(self):
        '''
        Processes the dataset to use it for training and validation.
        '''
        
        # add methods to formating the data to the  modeler.
    
    def categoryToDummy(self):
        # get dummy
        dummy = pd.get_dummies(self.extracted_data['category'])

        # merge df
        df_dummy = pd.merge(self.extracted_data, dummy, left_index=True, right_index=True)

        # delet no dummy cols
        del df_dummy['category']

        return df_dummy
    
    def split_train_test_validate(self):
        '''
        Split the dataset into train, validation and test
        '''
        
        # categorical variables list to split
        return list(self.data.columns[-6:])
        
        # full data categorized
        
        #X, X_test, y, y_test = train_test_split(X_data, y_data, test_size=0.2, train_size=0.8)
        #X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.25,train_size =0.75)
        
        #return (X, y), (X_train, y_train), (x_val, y_val)

# 3. Data Modeling

In [ ]:
class dataModeler(dataFormatter):
    def __init__(self):
        self.model = dataModeler.getModel()
        self.train_data = dataFormatter.train_data
        
    def getModel():
        '''
        Specifies a model to handle the categorization problem.
        # criar etapa de modelagem -métodos se for o caso
        # dar split no train e treinar
        
        
        '''

        model = None
        return model
        
    def writeModel(self):
        '''
        Exports a candidate model to a specified path available
        in the environment variable MODEL_PATH.
        '''

        # criar try: tenta salvar, se nao dá erro.
    
        return pickle.dump(self.model, open(os.getenv('MODEL_PATH'),'wb'))


# 4. Model Validation

In [ ]:
class modelValidator(dataModeler, dataFormatter):
    def __init__(self):
        self.model = self.readModel()
        self.validation_data = dataFormatter.validation_data
    
    def readModel(self):
        '''
        get the model selected to handle the categorization problem.
        '''
        
        # criar um try, se nao der entao dar um saveModel
        
        return pickle.load(open(os.getenv('MODEL_PATH'),'rb'))
        
    def validateModel(self):
        '''
        Generates metrics about the model accuracy (precision, recall, F1, etc.)
        for each category and exports them to a specified path available in the 
        environment variable METRICS_PATH.
        '''
        # try k-fold and variants.
        
        # should save metrics in METRICS_PATH
        f = open(os.getenv('METRICS_PATH'), "w")
        f.write("F1:95.0, Precision:87.5")
        f.close()
        pass

# 5. Model Prediction

In [ ]:
dataExtractor().extractData().head()

In [ ]:
dataExtractor().extractData().category.value_counts()

In [ ]:
dataFormatter().categoryToDummy()

In [ ]:
df = dataExtractor().extractData()

In [ ]:
dummy_df

In [ ]:
result = pd.merge(df, dummy_df, left_index=True, right_index=True)

In [ ]:
result.columns

In [ ]:
result.shape

In [ ]:
result = pd.merge(left, right, how="left", on=["key1", "key2"])